In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pivot").getOrCreate()

In [2]:
from pyspark.sql.functions import col

# Sample Data
data = [("John", "January", 100),
        ("John", "February", 150),
        ("Jane", "January", 200),
        ("Jane", "February", 250),
        ("Joe", "January", 300),
        ("Joe", "February", 350)]
columns = ["name", "month", "sales"]

df = spark.createDataFrame(data, columns)

df.show()


+----+--------+-----+
|name|   month|sales|
+----+--------+-----+
|John| January|  100|
|John|February|  150|
|Jane| January|  200|
|Jane|February|  250|
| Joe| January|  300|
| Joe|February|  350|
+----+--------+-----+



In [3]:
df.groupBy('name').count().show()

+----+-----+
|name|count|
+----+-----+
|John|    2|
|Jane|    2|
| Joe|    2|
+----+-----+



In [6]:
df.groupBy('name').pivot('month').sum('sales').show()

+----+--------+-------+
|name|February|January|
+----+--------+-------+
| Joe|     350|    300|
|John|     150|    100|
|Jane|     250|    200|
+----+--------+-------+



pivot for selected columns

In [7]:
df.groupBy('name').pivot('month',['January']).sum('sales').show()

+----+-------+
|name|January|
+----+-------+
| Joe|    300|
|John|    100|
|Jane|    200|
+----+-------+



In [8]:
df.groupBy('name').pivot('month',['January','February']).sum('sales').show()

+----+-------+--------+
|name|January|February|
+----+-------+--------+
| Joe|    300|     350|
|John|    100|     150|
|Jane|    200|     250|
+----+-------+--------+



In [9]:
pivot_df = df.groupBy('name').pivot('month',['January','February']).sum('sales')

pivot_df.show()

+----+-------+--------+
|name|January|February|
+----+-------+--------+
| Joe|    300|     350|
|John|    100|     150|
|Jane|    200|     250|
+----+-------+--------+



UNPIVOT --> this is not available so we need to use stack method

In [12]:
from pyspark.sql.functions import expr
unpivot_df = pivot_df.selectExpr("name","stack(2,'January',January,'February',February) as (month,sales)")

unpivot_df.show()

+----+--------+-----+
|name|   month|sales|
+----+--------+-----+
| Joe| January|  300|
| Joe|February|  350|
|John| January|  100|
|John|February|  150|
|Jane| January|  200|
|Jane|February|  250|
+----+--------+-----+

